<a href="https://colab.research.google.com/github/t-aridi/DS2002-002-Assignments/blob/main/superhero_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Superhero Database Assignment (SQLite & Google Colab)

## Objective
In this assignment, you will design and implement a **Superhero Database** using SQLite in Google Colab. This database will store superheroes, villains, and their battles. You will write SQL queries to analyze battle outcomes and relationships between heroes and villains.

### Group Size:
- 2 to 3 students per group.

### Tools Required:
- **Google Colab**
- **SQLite (built into Python)**
- **CSV files (provided for import)**

## Step 1: Setting Up Your SQLite Database

### Tasks:
1. Initialize SQLite in Google Colab.
2. Create a database named `superhero_db.sqlite`.
3. Define the database schema, ensuring it includes the following tables:
   - `heroes`: Stores superhero names, powers, and teams.
   - `villains`: Stores villains and their evil plans.
   - `battles`: Tracks battles between heroes and villains, including the outcome.
4. Implement primary keys and foreign keys where appropriate.

✅ **Checkpoint:** Ensure that the database and tables have been successfully created.

In [1]:
import sqlite3

# Connect to (or create) the SQLite database file
conn = sqlite3.connect('superhero_db.sqlite')
cursor = conn.cursor()

# Create the heroes table (storing names, powers, and teams)
cursor.execute('''
CREATE TABLE IF NOT EXISTS heroes (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    power TEXT,
    team TEXT
)
''')

# Create the villains table (storing names and evil plans)
cursor.execute('''
CREATE TABLE IF NOT EXISTS villains (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    evil_plan TEXT
)
''')

# Create the battles table with foreign keys referencing heroes and villains
cursor.execute('''
CREATE TABLE IF NOT EXISTS battles (
    id INTEGER PRIMARY KEY,
    hero_id INTEGER,
    villain_id INTEGER,
    outcome TEXT,
    FOREIGN KEY (hero_id) REFERENCES heroes (id),
    FOREIGN KEY (villain_id) REFERENCES villains (id)
)
''')

conn.commit()
tables = ['heroes', 'villains', 'battles']
for table in tables:
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name=?", (table,))
    result = cursor.fetchone()
    if result:
        print(f"Table '{table}' exists.")
    else:
        print(f"Error: Table '{table}' does not exist in the database.")
conn.close()

Table 'heroes' exists.
Table 'villains' exists.
Table 'battles' exists.


## Step 2: Import Data from CSV Files

### Tasks:
1. Download the provided CSV files.
2. Upload them to Google Colab.
3. Write a script to import data from these files into the appropriate tables.
4. Verify that the data has been successfully inserted.

✅ **Checkpoint:** Ensure that each table contains the expected data from the CSV files.

In [2]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('superhero_db.sqlite')

# Read CSV files with the revised columns
heroes_df = pd.read_csv('heroes.csv')
villains_df = pd.read_csv('villains.csv')
battles_df = pd.read_csv('battles.csv')

# Import data into the respective tables
heroes_df.to_sql('heroes', conn, if_exists='append', index=False)
villains_df.to_sql('villains', conn, if_exists='append', index=False)
battles_df.to_sql('battles', conn, if_exists='append', index=False)

# Verify data insertion by counting rows in each table
heroes_count = pd.read_sql_query('SELECT COUNT(*) AS count FROM heroes', conn).iloc[0]['count']
villains_count = pd.read_sql_query('SELECT COUNT(*) AS count FROM villains', conn).iloc[0]['count']
battles_count = pd.read_sql_query('SELECT COUNT(*) AS count FROM battles', conn).iloc[0]['count']

print("Heroes:", heroes_count)
print("Villains:", villains_count)
print("Battles:", battles_count)

conn.close()

Heroes: 10
Villains: 10
Battles: 30


## Step 3: Querying the Database

### Tasks:
Write SQL queries to answer the following:
1. Retrieve all superheroes and their powers.
2. Find all battles where the hero won.
3. List all villains and their evil plans.
4. Determine which superhero has fought the most battles.

✅ **Checkpoint:** Ensure that all queries return meaningful results from the database.

In [3]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('superhero_db.sqlite')

# 1. Retrieve all superheroes and their powers.
query1 = "SELECT name, power FROM heroes"
superheroes = pd.read_sql_query(query1, conn)
print("Superheroes and their powers:")
print(superheroes)

# 2. Find all battles where the hero won.
# (Assumes the outcome field in battles.csv contains 'win' for victories.)
query2 = "SELECT * FROM battles WHERE LOWER(outcome) LIKE 'Hero Wins'"
battles_won = pd.read_sql_query(query2, conn)
print("\nBattles where the hero won:")
print(battles_won)

# 3. List all villains and their evil plans.
query3 = "SELECT name, evil_plan FROM villains"
villains = pd.read_sql_query(query3, conn)
print("\nVillains and their evil plans:")
print(villains)

# 4. Determine which superhero has fought the most battles.
query4 = """
SELECT h.name, COUNT(b.id) AS battle_count
FROM heroes h
JOIN battles b ON h.id = b.hero_id
GROUP BY h.id
ORDER BY battle_count DESC
LIMIT 1
"""
most_active_hero = pd.read_sql_query(query4, conn)
print("\nSuperhero who fought the most battles:")
print(most_active_hero)

conn.close()

Superheroes and their powers:
             name               power
0      Spider-Man        Web-slinging
1        Iron Man       Powered Armor
2          Batman    Genius Detective
3        Superman      Super Strength
4            Thor      God of Thunder
5    Wonder Woman  Amazonian Strength
6   Black Panther    Enhanced Agility
7       The Flash         Super Speed
8            Hulk      Super Strength
9  Doctor Strange         Mystic Arts

Battles where the hero won:
   id  hero_id  villain_id    outcome
0   2        9           3  Hero Wins
1   5        1           7  Hero Wins
2   7        9           8  Hero Wins
3   8        3           2  Hero Wins
4   9        7           5  Hero Wins
5  13        8           9  Hero Wins
6  20        3           4  Hero Wins
7  29        3           5  Hero Wins
8  30        5           5  Hero Wins

Villains and their evil plans:
           name                           evil_plan
0  Green Goblin                  Terrorize New York
1      

## Step 4: Add a Custom Feature

### Tasks:
1. Design and implement an additional table related to superheroes or villains.
2. Populate it with data.
3. Write at least one query to retrieve meaningful insights from the new table.

**Examples of Custom Features:**
- Sidekicks for superheroes
- Villain hideouts
- Battle locations
- Superhero equipment

✅ **Checkpoint:** Ensure that the new table and queries function correctly.

In [4]:
import sqlite3

# Connect to the database (or create it if it doesn't exist)
conn = sqlite3.connect('superhero_db.sqlite')
cursor = conn.cursor()

# Task 1: Create a new table for villain hideouts
create_table_query = '''
CREATE TABLE IF NOT EXISTS VillainHideouts (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    villain_name TEXT NOT NULL,
    hideout_location TEXT NOT NULL,
    hideout_type TEXT,
    description TEXT
);
'''
cursor.execute(create_table_query)
conn.commit()

# Task 2: Populate the VillainHideouts table with data
insert_data_query = '''
INSERT INTO VillainHideouts (villain_name, hideout_location, hideout_type, description)
VALUES
    ('Joker', 'Abandoned Amusement Park', 'Urban Ruin', 'A dilapidated park with hidden passages.'),
    ('Lex Luthor', 'Luminous Tower', 'Corporate High-tech', 'A skyscraper outfitted as a secret laboratory.'),
    ('Green Goblin', 'Industrial Warehouse', 'Underground', 'An abandoned warehouse that conceals secret tunnels.'),
    ('Doctor Octopus', 'Dockside Laboratory', 'Seaside Facility', 'A lab near the docks with advanced tech.'),
    ('Magneto', 'Eastern Fortress', 'Remote Base', 'A fortified base located in a remote area.');
'''
cursor.execute(insert_data_query)
conn.commit()

# Task 3: Retrieve meaningful insights – Count hideouts by hideout type
query = '''
SELECT hideout_type, COUNT(*) AS number_of_hideouts
FROM VillainHideouts
GROUP BY hideout_type;
'''
cursor.execute(query)
results = cursor.fetchall()

print("Hideout Types and their Counts:")
for row in results:
    print(f"Hideout Type: {row[0]}, Count: {row[1]}")

conn.close()


Hideout Types and their Counts:
Hideout Type: Corporate High-tech, Count: 1
Hideout Type: Remote Base, Count: 1
Hideout Type: Seaside Facility, Count: 1
Hideout Type: Underground, Count: 1
Hideout Type: Urban Ruin, Count: 1


## Bonus Challenge (+5 Points)

### Tasks:
1. Enhance your project by adding a visual or interactive component.

**Options:**
- Visualize battles using Matplotlib (e.g., a bar chart showing how many battles each superhero has fought).
- Make it interactive (e.g., allow users to input a superhero name and display their battles).

✅ **Checkpoint:** If attempting the bonus, ensure that your visualization or interactive component works correctly.

In [5]:
# @title
import sqlite3
import ipywidgets as widgets
from IPython.display import display

# Connect to the existing database (which already has battles, heroes, and villains tables)
conn = sqlite3.connect('superhero_db.sqlite')
cursor = conn.cursor()

# Function to retrieve battles for a given superhero name by joining the battles, heroes, and villains tables
def get_superhero_battles(hero_name):
    query = '''
    SELECT b.id, h.name AS hero_name, v.name AS villain_name, b.outcome
    FROM battles b
    JOIN heroes h ON b.hero_id = h.id
    JOIN villains v ON b.villain_id = v.id
    WHERE h.name = ?
    '''
    cursor.execute(query, (hero_name,))
    return cursor.fetchall()

# Callback function for the interactive widget
def on_button_clicked(b):
    hero = text_input.value.strip()
    if not hero:
        output.value = "<p>Please enter a superhero name.</p>"
        return

    battles = get_superhero_battles(hero)
    if battles:
        output_html = f"<h4>Battles for {hero}:</h4><ul>"
        for battle in battles:
            output_html += f"<li>Battle ID: {battle[0]}, Opponent: {battle[2]}, Outcome: {battle[3]}</li>"
        output_html += "</ul>"
    else:
        output_html = f"<p>No battles found for <em>{hero}</em>.</p>"

    output.value = output_html

# Create interactive widgets: a text input field and a button
text_input = widgets.Text(
    value='',
    placeholder='Enter a superhero name...',
    description='Superhero:',
    disabled=False
)

button = widgets.Button(
    description='Show Battles',
    tooltip='Click to display battles for the superhero'
)
button.on_click(on_button_clicked)

# HTML widget for displaying output results
output = widgets.HTML()

# Display the interactive component vertically
display(widgets.VBox([text_input, button, output]))

## Submission Guidelines

Each group must submit a **Google Colab Notebook (.ipynb)** containing:
- ✅ **Database schema (DDL)**
- ✅ **CSV imports & data insertion (DML)**
- ✅ **SQL queries & results**
- ✅ **Custom feature (bonus if applicable)**

## Grading Criteria (Total: 25 Points)

| Section                 | Points |
|-------------------------|--------|
| Database Setup          | 5      |
| Data Importing         | 10     |
| Querying the Data      | 10     |
| Extra Feature          | 5      |
| Bonus Challenge (Optional) | +5  |

### Final Notes:
- Be **creative** when designing superheroes and villains.
- **Comment your code** to explain its functionality.
- **Have fun** building your superhero universe!